# Skill lab: Comparing classifiers


In this assignment you will apply the statistical tools we learned to a machine learning task of comparing performance of two classifiers.

By the end of this lab you will know
- How to implement a k-nearest neighbor classifier.
- How to perform a k-fold cross validation.
- How to find confidence intervals for a classiifer performance based on a sample.
- How to statistically compare performance of two classsifiers.



You need to perform the following seven tasks:
1. Compute the accuracy of the Naive Bayes classifier based on the holdout estimation. Next, compute the confidence interval for accuracy at the confidence level 0.90.
2. Break the original dataset into 10 folds for cross-validation of Naive Bayes classifier. Obtain classification results from 10 cross-validation experiments.
3. Implement the Nearest Neighbors classifier. 
4. Use it to find the accuracy based on the holdout estimation. Compute the confidence interval at the confidence level 0.90. 
5. Generate the same 10-folds from a dataset with all numeric attributes and obtain classification results using the k-NN classifer. 
6. Test the hypothesis that two classifers have a diferent performance at significance level 0.05.
7. Use the best classifier to predict the evaluation score of several instructors that you know.

Feel free to use any programming tools available: pandas, plain python, numpy or anything else. 

**You are not allowed to use sklearn or any other python library that already includes the implementation of all these tasks**.


## Dataset

The dataset for this lab contains about 460 anonymized student evaluations collected at the University of Texas at Austin, and used in the following publication: "Beauty in the Classroom: Instructors' Pulchritude and Putative Pedagogical Productivity". You can learn how the data was collected and the meaning of various data attributes following [THIS LINK](https://chance.amstat.org/2013/04/looking-good/).

We use a subset of attributes. This smaller subset of the original data is included in the repository. We want to build a classifier that &mdash; based on these attributes &mdash; will predict the evaluation result for each instructor: good (&ge; 4) or bad (<4). 

In [14]:
data_file = "SStudentEvaluations.csv"

In [15]:
import pandas as pd

data = pd.read_csv(data_file)
print(data.columns)
print(data.dtypes)

Index(['rank', 'ethnicity', 'gender', 'language', 'age', 'bty_avg',
       'eval_categorical'],
      dtype='object')
rank                 object
ethnicity            object
gender               object
language             object
age                   int64
bty_avg             float64
eval_categorical     object
dtype: object


In [16]:
display(data)

,rank,ethnicity,gender,language,age,bty_avg,eval_categorical
0,tenure track,minority,female,english,36,5.000,good
1,tenure track,minority,female,english,36,5.000,bad
2,tenure track,minority,female,english,36,5.000,bad
3,tenure track,minority,female,english,36,5.000,good
4,tenured,not minority,male,english,59,3.000,good
...,...,...,...,...,...,...,...
458,tenure track,not minority,male,english,32,6.833,good
459,tenure track,minority,female,non-english,42,5.333,bad
460,tenure track,minority,female,non-english,42,5.333,bad
461,tenure track,minority,female,non-english,42,5.333,bad


First of all, we will shuffle the data. We use a seeded randomization &mdash; so we can obtain reproducible results (needed for testing of your work).

In [17]:
data = data.sample(frac = 1, random_state=1)    # shuffling the data before performing any validation
data.head()

,rank,ethnicity,gender,language,age,bty_avg,eval_categorical
331,tenured,not minority,male,english,64,2.333,bad
101,tenured,not minority,female,english,46,4.333,good
192,tenured,not minority,male,english,54,2.333,good
66,teaching,not minority,male,english,37,4.333,bad
327,tenured,not minority,male,english,64,2.333,bad


### Holdout estimation
That is how we can divide the dataset into training and testing sets in proportion of ~ 2:1:

In [18]:
# Select ratio
ratio = 0.66
 
total_rows = data.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
data_train = data[0:train_size]
data_test = data[train_size:]

In [19]:
data_train.shape[0]

305

In [20]:
data_test.shape[0]

158

## Naive Bayes classifier

Below we provide our implementation of the first classifier: Naive Bayes.

We have a mix of cathegorical and numeric attributes. We will produce counts and probabilities for cathegorical attributes. We will also precompute the mean and standard deviation for the numeric attributes which we will later use with the normal distribution probability density function (PDF) to compute the contribution of numeric attributes. 

Here is an implementation of the PDF:

In [21]:
from math import *

def normal_pdf(x, stat):
    """
    :param x: a variable
    :param mean: µ - the expected value or average from M samples
    :param stdev: σ - standard deviation
    :return: Gaussian (Normal) Density function.
    N(x; µ, σ) = (1 / 2πσ) * (e ^ (x–µ)^2/-2σ^2
    """
    mean, stdev = stat
    variance = stdev ** 2
    exp_squared_diff = (x - mean) ** 2
    exp_power = -exp_squared_diff / (2 * variance)
    exponent = e ** exp_power
    denominator = ((2 * pi) ** .5) * stdev
    normal_prob = exponent / denominator
    return normal_prob

Here is our counting function:

In [22]:
def produce_counts (train_set, column, results):
    # counter = 5
    col_idx = col_name_to_col_idx [column]
    for tup in train_set.itertuples():
        val = tup[col_idx]
        class_label = tup[7]
        prev = results [class_label][column]

        if val not in prev.keys():
            prev[val] = 0
        prev[val] += 1    

Based on these counts, we can pre-compute conditional probabilities for all combinations of cathegorical attributes and class labels:

In [23]:
def produce_probabilities(counts, results, class_label, total):
    for col in counts[class_label].keys():
        results[class_label][col] = {} 
        cardinality = len(counts[class_label][col].keys())
        
        for val in counts[class_label][col].keys():
            results[class_label][col][val] = (counts[class_label][col][val] + 1)/(total + cardinality)      
                

The classification algorithm that classifies all the records in the *test_set*, based on the data in the *train_set*. 

The output is the list of classification results in form of a tuple (*classified*, *actual*), where *classified* is a class label obtained by our classification, and *actual* is the actual label of this record in the test set.

In [24]:
col_name_to_col_idx = {"rank":1, "ethnicity":2, "gender":3, "language":4 }
idx_to_col_name = {1:"rank", 2: "ethnicity", 3: "gender", 4: "language"}

def naive_bayes_classify (train_set, test_set):  
    counts = {"good": {"rank":{}, "ethnicity":{}, "gender":{}, "language":{}}, "bad":{"rank":{}, "ethnicity":{}, "gender":{}, "language":{}} }   

    total_good  = train_set.groupby("eval_categorical").size()["good"]
    total_bad = train_set.groupby("eval_categorical").size()["bad"]
    priors = {"good":total_good/(total_good+total_bad), "bad":total_bad/(total_good+total_bad) }

    for col in col_name_to_col_idx.keys():
        produce_counts(train_set, col, counts)   
    # print(counts)
    
    probs = {"good":{}, "bad":{}}
    produce_probabilities (counts,  probs, "good", total_good)
    produce_probabilities (counts,  probs, "bad", total_bad)
    # print(probs)

    # means and std for normal distribution of numeric parameters
    data_good = train_set[train_set["eval_categorical"]== "good"]
    data_bad = train_set[train_set["eval_categorical"]== "bad"]

    stats = {"good":{"age":(data_good["age"].mean(), data_good["age"].std(ddof=1)), 
                 "bty_avg":(data_good["bty_avg"].mean(), data_good["bty_avg"].std(ddof=1)) },
        "bad":{"age":(data_bad["age"].mean(), data_bad["age"].std(ddof=1)), 
                 "bty_avg":(data_bad["bty_avg"].mean(), data_bad["bty_avg"].std(ddof=1)) }}
    #print(stats)
    
    results = []
    for tup in test_set.itertuples():
        class_label = tup[7]
        prob_good = log (priors["good"]) 
        for k in col_name_to_col_idx.keys():
            prob_good += log (probs["good"][k][tup[col_name_to_col_idx[k]]]) 
        prob_good += normal_pdf(tup[5], stats["good"]["age"])
        prob_good += normal_pdf(tup[6], stats["good"]["bty_avg"])
        # print ("good:", prob_good)

        prob_bad = log (priors["bad"]) 
        for k in col_name_to_col_idx.keys():
            prob_bad += log (probs["bad"][k][tup[col_name_to_col_idx[k]]]) 
        prob_bad += normal_pdf(tup[5], stats["bad"]["age"])
        prob_bad += normal_pdf(tup[6], stats["bad"]["bty_avg"])
        # print ("bad:", prob_bad)

        classified_as = "good"
        if prob_bad > prob_good:
            classified_as = "bad"
        
        results += [(classified_as, class_label )]    
    return results

Let's run the classifier using the training and testing parts we obtained in the holdout section.

In [25]:
class_results = naive_bayes_classify(data_train, data_test)
print(class_results[:5])

correct_count = 0
for r in range(len(class_results)):
    if class_results[r][0] == class_results[r][1]:
        correct_count+= 1
print ("Accuracy:", correct_count/len(class_results))

[('good', 'good'), ('good', 'good'), ('good', 'bad'), ('good', 'good'), ('bad', 'good')]
Accuracy: 0.569620253164557


<div style="background-color:yellow;">
    <h3>Task 1. Generate confidence interval for accuracy of the Naive Bayes</h3>    
</div>
You can write the code, or use the tables manually.

In [26]:
import numpy as np
import scipy.stats as stats

acc = correct_count/len(class_results)

n = 305

std_err = np.sqrt((acc *(1-acc)) / n)

confidence = 0.90

crit_val = stats.norm.ppf((1+confidence) / 2)

margin_err = crit_val * std_err

lower = acc - margin_err
upper = acc + margin_err

print(lower)
print(upper)


0.5229869733623871
0.6162535329667269


**Answer**: The confidence interval for the performance of the Naive Bayes classifier is: [0.522, 0.616]

### Cross-validation

We want to test the performance of two classifiers on different datasets &mdash; to get the mean of the paired difference. To create several traning/testing subsets we will use 10-fold cross-validation: we will divide our original dataset into 10 approximately equal parts (folds) and use 9 out of 10 folds for training and 1 fold for testing. Hence, the total number of performance experiments will be 10.

<div style="background-color:yellow;">
    <h3>Task 2. Perform the 10-fold cross-validation with Naive Bayes</h3>    
</div>

Generate 10 equal non-overlapping subsets of data and store them in the list of pandas data frames called *folds*:

Implement the loop where you obtain classification results for each of the folds. Store these results in the list *nb_accuracies* for future use.

In [27]:
# We want 10 folds of the same size, 46 each (don't include the last datapoints)
k = 10
fold_size = 46

#Calculate total number of datapoints
total_data = k*fold_size

# Create an array for the folds to be stored into
folds = []

# Split data into data folds
# From range i to i+46, add those datasets into the fold given
for i in range(k):
    start = i * fold_size
    end = start + fold_size
    fold = data.iloc[start:end]
    folds.append(fold)
    print("Fold", i, "size:", folds[i].shape[0])

Fold 0 size: 46
Fold 1 size: 46
Fold 2 size: 46
Fold 3 size: 46
Fold 4 size: 46
Fold 5 size: 46
Fold 6 size: 46
Fold 7 size: 46
Fold 8 size: 46
Fold 9 size: 46


In [28]:
# Initialize list to store classifications
nb_accuracies = []

# Iterate over each fold
for fold_index, fold_data in enumerate(folds):
    # Perform Naive Bayes classification and calculate accuracy
    correct = sum(1 for classified_as, actual in naive_bayes_classify(pd.concat([folds[i] for i in range(k) if i != fold_index]), fold_data) if classified_as == actual)
    acc = correct / len(fold_data)
    
    # Store this in the list
    nb_accuracies.append(acc)

# print
print(nb_accuracies)


[0.5652173913043478, 0.4782608695652174, 0.5217391304347826, 0.5652173913043478, 0.5217391304347826, 0.6956521739130435, 0.5434782608695652, 0.6086956521739131, 0.5434782608695652, 0.5]


For comparison &mdash; here are our results: 0.5652173913043478, 0.4782608695652174, 0.5217391304347826, 0.5652173913043478, 
    0.5217391304347826, 0.6956521739130435, 0.5434782608695652, 0.6086956521739131, 0.5434782608695652, 0.5

## Nearest Neighbors classifier (k-NN)

This classifier assigns a class to a given record based on the class labels of *k* labeled records that are closest to it. The closest samples are selected based on a distance metric, then the neighbors vote and the majority class is assigned to a record in question.

The value of *k* indicates the number of closest neighbors used to classify the test record. The value of *k* is non-parametric and a general rule of thumb in choosing the initial value of k is: k = sqrt(N)/2, where N stands for the number of samples in the training dataset. Another hint is to keep the value of k odd, so that there is no tie when choosing between two classes.

For our dataset the size of the training set will be about 9 * 46 = 414, and sqrt(414)/2 is ~ 11. We will use k=11 nearest neighbors for our classification.

### Categorical to numeric (binary)
To use distance metrics we must convert the categorical attributes to numeric. The most common method is to convert a categorical attribute into a set of binary attributes, such that for each categorical value there is a separate column, and the value in this column is either 0 or 1. This is called a "one hot encoding".

One hot encoding for categorical columns:

In [29]:
ohe_rank = pd.get_dummies(data["rank"], dtype=int)
pd.concat([ohe_rank, data["rank"]], axis=1, sort=False).head()

,teaching,tenure track,tenured,rank
331,0,0,1,tenured
101,0,0,1,tenured
192,0,0,1,tenured
66,1,0,0,teaching
327,0,0,1,tenured


In [30]:
ohe_ethnicity = pd.get_dummies(data["ethnicity"], dtype=int)
pd.concat([ohe_ethnicity, data["ethnicity"]], axis=1, sort=False).head()

,minority,not minority,ethnicity
331,0,1,not minority
101,0,1,not minority
192,0,1,not minority
66,0,1,not minority
327,0,1,not minority


In [31]:
ohe_gender = pd.get_dummies(data["gender"], dtype=int)
pd.concat([ohe_gender, data["gender"]], axis=1, sort=False).head()

,female,male,gender
331,0,1,male
101,1,0,female
192,0,1,male
66,0,1,male
327,0,1,male


In [32]:
ohe_language = pd.get_dummies(data["language"], dtype=int)
pd.concat([ohe_language, data["language"]], axis=1, sort=False).head()

,english,non-english,language
331,1,0,english
101,1,0,english
192,1,0,english
66,1,0,english
327,1,0,english


Now we create a dataset where all the cathegorical attributes are replaced by the binary columns. This dataset is called *num_data* and it will be used in the k-NN classification.

In [33]:
num_data = pd.concat([ohe_rank, ohe_ethnicity, ohe_gender, ohe_language, data[["age","bty_avg","eval_categorical"]]], axis=1, sort=False)
num_data.head()

,teaching,tenure track,tenured,minority,not minority,female,male,english,non-english,age,bty_avg,eval_categorical
331,0,0,1,0,1,0,1,1,0,64,2.333,bad
101,0,0,1,0,1,1,0,1,0,46,4.333,good
192,0,0,1,0,1,0,1,1,0,54,2.333,good
66,1,0,0,0,1,0,1,1,0,37,4.333,bad
327,0,0,1,0,1,0,1,1,0,64,2.333,bad


Now all the data in num_data is numeric, and we can use the Euclidean distance to compute the distance between the records.

### Common scale
You can see that the absolute values of different attributes are on different scales, and we better bring them all to the same interval between 0 and 1, since otherwise the difference in age will dominate an overall distance between two records.

We transform numeric columns to a standard scale 0-1 using the following formula: x<sub>scaled</sub>=(x-min)/(max-min)

In [34]:
# apply normalization techniques to column age 
column = 'age'
num_data[column] = (num_data[column] - num_data[column].min()) / (num_data[column].max() - num_data[column].min())     

In [35]:
# apply normalization techniques to column bty_avg 
column = 'bty_avg'
num_data[column] = (num_data[column] - num_data[column].min()) / (num_data[column].max() - num_data[column].min())     
  
# view normalized data 
display(num_data) 

,teaching,tenure track,tenured,minority,not minority,female,male,english,non-english,age,bty_avg,eval_categorical
331,0,0,1,0,1,0,1,1,0,0.795455,0.102462,bad
101,0,0,1,0,1,1,0,1,0,0.386364,0.410154,good
192,0,0,1,0,1,0,1,1,0,0.568182,0.102462,good
66,1,0,0,0,1,0,1,1,0,0.181818,0.410154,bad
327,0,0,1,0,1,0,1,1,0,0.795455,0.102462,bad
...,...,...,...,...,...,...,...,...,...,...,...,...
255,0,0,1,0,1,0,1,1,0,0.522727,0.230769,good
72,0,0,1,0,1,0,1,1,0,0.295455,0.487077,good
396,1,0,0,0,1,0,1,1,0,0.363636,0.256308,good
235,0,0,1,0,1,0,1,1,0,0.727273,0.487077,good


### Holdout for the numeric dataset
Divide the dataset into training and testing sets in proportion of 2:1.

In [36]:
# Select ratio
ratio = 0.66
 
total_rows = num_data.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
num_data_train = num_data[0:train_size]
num_data_test = num_data[train_size:]




Now you have the input dataset for the k-NN classification.

<div style="background-color:yellow;">
    <h3>Task 3. Implement the k-NN classifier</h3>    
</div>

Note that this is a "lazy" classifier and nothing can be precomputed. Both the training and the test sets are used only during classification.

The output of a classifier should be the list of classification results in form of a tuple (*classified*, *actual*), where *classified* is a class label obtained by our classification, and *actual* is the actual label of this record in the test set.

In [52]:

# Calulates the euclidiean distance between 2 points
def calculate_distance(instance1, instance2, col_to_index):
    distance = 0
    for col, idx in col_to_index.items():
        # Square it, so that it is always positive
        distance += (instance1[col] - instance2[col]) ** 2
    # Return square root of squared number so that it is positive
    return distance ** 0.5

# Similar to produce counts method written for Bayes Classifier
# Adds the columns up for each category
def produce_counts(train_set, column, results):
    for _, row in train_set.iterrows():
        val = row[column]
        class_label = row["eval_categorical"]
        if column not in results[class_label]:
            results[class_label][column] = 0
        results[class_label][column] += val
        
# Selects the most popular our of the given set
def majority_vote(neighbors):
    class_votes = {}
    for neighbor in neighbors:
        label = neighbor[0]["eval_categorical"]
        if label in class_votes:
            class_votes[label] += 1
        else:
            class_votes[label] = 1
    return max(class_votes, key=class_votes.get)


def knn_classify(train_set, test_set, knn):
    #Indexing the column names for easier use
    col_name_to_col_idx = {
        "teaching": 1, "tenure track": 2, "tenured": 3, "minority": 4, "not minority": 5,
        "female": 6, "male": 7, "english": 8, "non-english": 9, "age": 10, "bty_avg": 11
    }
    idx_to_col_name = {v: k for k, v in col_name_to_col_idx.items()}
    
    # Initialize for counts
    results = {"good": {}, "bad": {}}
    
    # Add up the values
    for col in col_name_to_col_idx:
        produce_counts(train_set, col, results)
    #Initialize for predictions    
    predictions = []
    
    # Go through each to calculate neighbors
    for index, row in test_set.iterrows():
        distances = []
        for index_train, row_train in train_set.iterrows():
            distance = calculate_distance(row, row_train, col_name_to_col_idx)
            distances.append((row_train, distance))
        distances.sort(key=lambda x: x[1])
        neighbors = distances[:knn]
        prediction = majority_vote(neighbors)
        predictions.append(prediction)
        
    return predictions


<div style="background-color:yellow;">
    <h3>Task 4. Generate the confidence interval for the k-NN accuracy </h3>    
</div>
This is based on the holdout estimation. 
Run your classifier, obtain the accuracy of the sample, and then produce a confidence interval. You can write the code, or use the tables manually.

In [53]:


class_results = knn_classify(num_data_train, num_data_test, 11)
print(class_results[:5])

# Calculating accuracy
correct = sum(1 for predicted, actual in zip(class_results, num_data_test["eval_categorical"]) if predicted == actual)
accuracy = correct / len(class_results)
print("Accuracy:", accuracy)

# Computing confidence interval
from scipy.stats import binom

# Assuming 90% confidence level
confidence_level = 0.90

# Compute correct predictions
num_successes = correct

# total number of predictions
sample_size = len(class_results)

# Compute confidence interval 
lower_bound, upper_bound = binom.interval(confidence_level, sample_size, accuracy)

print("Confidence Interval:")
print("Lower Bound:", lower_bound)
print("Upper Bound:", upper_bound)


# TODO: classify and compute accuracy


['good', 'good', 'good', 'good', 'good']
Accuracy: 0.5569620253164557
Confidence Interval:
Lower Bound: 78.0
Upper Bound: 98.0


Our accuracy was: 0.5569620253164557

**Your answer**: The confidence interval for the performance of the k-NN classifier is: [78,98]


<div style="background-color:yellow;">
    <h3>Task 5. Perform the 10-fold cross-validation with k-NN</h3>    
</div>


Generate 10 equal non-overlapping subsets of numeric data from the numeric dataset and store them in the list of pandas data frames called *num_folds*:

In [58]:
# We want 10 folds of the same size, 46 each (don't include the last datapoints)
k = 10
fold_size = 46

#Calculate total number of datapoints
total_data = k*fold_size

# Create an array for the folds to be stored into
num_folds = []

# Split data into data folds
# From range i to i+46, add those datasets into the fold given
for i in range(k):
    start = i * fold_size
    end = start + fold_size
    fold = num_data.iloc[start:end]
    num_folds.append(fold)
    print("Fold", i, "size:", num_folds[i].shape[0])

Fold 0 size: 46
Fold 1 size: 46
Fold 2 size: 46
Fold 3 size: 46
Fold 4 size: 46
Fold 5 size: 46
Fold 6 size: 46
Fold 7 size: 46
Fold 8 size: 46
Fold 9 size: 46


Implement the loop to perform 10-fold cross-validation. Store the classification results in the list *knn_accuracies* for future use.

In [60]:
k = 10
fold_size = 46
knn_accuracies = []

for i in range(k):
    # Do the classification on the data
    class_results = knn_classify(num_data_train, num_data_test, 11)
    
    # Find the accuracy for each fold
    correct = sum(1 for predicted, actual in zip(class_results, test_set["eval_categorical"]) if predicted == actual)
    accuracy = correct / len(class_results)
    knn_accuracies.append(accuracy)
    
    print("Accuracy for fold", i, ":", accuracy)

print("Mean accuracy:", np.mean(knn_accuracies))
print("Standard deviation of accuracy:", np.std(knn_accuracies))


Accuracy for fold 0 : 0.189873417721519
Accuracy for fold 1 : 0.13924050632911392
Accuracy for fold 2 : 0.14556962025316456
Accuracy for fold 3 : 0.12658227848101267
Accuracy for fold 4 : 0.17721518987341772
Accuracy for fold 5 : 0.17721518987341772
Accuracy for fold 6 : 0.1518987341772152
Accuracy for fold 7 : 0.17088607594936708
Accuracy for fold 8 : 0.15822784810126583
Accuracy for fold 9 : 0.12025316455696203
Mean accuracy: 0.15569620253164557
Standard deviation of accuracy: 0.02196120452265503


In [61]:
import numpy as np

k = 10
knn_accuracies = []

for i in range(k):
    test_set = num_folds[i]
    train_set = pd.concat([fold for j, fold in enumerate(num_folds) if j != i])
    
    # Perform classification
    class_results = knn_classify(train_set, test_set, 11)
    
    # Calculate accuracy
    correct = sum(1 for predicted, actual in zip(class_results, test_set["eval_categorical"]) if predicted == actual)
    accuracy = correct / len(class_results)
    knn_accuracies.append(accuracy)
    
    print("Accuracy for fold", i, ":", accuracy)



Accuracy for fold 0 : 0.6086956521739131
Accuracy for fold 1 : 0.6086956521739131
Accuracy for fold 2 : 0.5217391304347826
Accuracy for fold 3 : 0.6086956521739131
Accuracy for fold 4 : 0.6739130434782609
Accuracy for fold 5 : 0.5869565217391305
Accuracy for fold 6 : 0.6521739130434783
Accuracy for fold 7 : 0.5434782608695652
Accuracy for fold 8 : 0.6956521739130435
Accuracy for fold 9 : 0.6304347826086957
Mean accuracy: 0.6130434782608696
Standard deviation of accuracy: 0.051260113576311286


Our results were:
0.6304347826086957, 0.717391304347826, 0.5, 0.6304347826086957, 0.5869565217391305, 0.6304347826086957, 
0.717391304347826, 0.6521739130434783, 0.6956521739130435, 0.6739130434782609

<div style="background-color:yellow;">
    <h3>Task 6. Compare performance of two classifiers</h3>    
</div>

Based on the paired results stored in lists *nb_accuracies* and *knn_accuracies*, test the hypothesis that the two classifiers do not have the same performance at a significance level 0.05. Recall that we need to use the t-ditribution for the mean of differences. Again, you can either implement the computation or use the tables manually.

**If you are not writing the code, please clearly explain all the steps of your computation**.

In [63]:
from scipy.stats import t

# Subtract each accuracy value from the other corresponding value of the same index
differences = [nb_accuracy - knn_accuracy for nb_accuracy, knn_accuracy in zip(nb_accuracies, knn_accuracies)]

# Find the mean and standard deviation of each
mean_difference = np.mean(differences)
std_difference = np.std(differences, ddof=1)  # ddof=1 for sample standard deviation

# Find the t-value
n = len(differences)
t_statistic = mean_difference / (std_difference / np.sqrt(n))

# DOF
df = n - 1

# 2-tailed significance test and 0.05 significance level
alpha = 0.05
critical_value = t.ppf(1 - alpha / 2, df)

# Print results
print("T-Statistic:", t_statistic)
print("Critical Value:", critical_value)

# Compare values and decide the hypothesis
if np.abs(t_statistic) > critical_value:
    print("We reject the null hypothesis. The 2 classifiers have significantly different performance")
else:
    print("We failed to reject the null hypothesis. There is no significant difference in performance between the 2 classifiers.")


T-Statistic: -1.9995429093758168
Critical Value: 2.2621571627409915
We failed to reject the null hypothesis. There is no significant difference in performance between the 2 classifiers.


<div style="background-color:yellow;">
    <h3>Task 7. Use the best classifier</h3>    
</div>
Which classifier is significantly better? 

Use it to predict the evaluation results for instructors that you know.
Now you can use the entire dataset as a training set.

Did the predicted class labels correspond to your own evaluations? 

Discuss all these questions and add any notes about this lab in a separate cell below.

Neither classifer is significantly better than the other. Shown above, the calculated t-statistic that we found using our 2 classifiers was -1.9995, and needed to be 2.262 in order to prove that one was better than the other. We cannot accept the null hypothesis, but we still cannot accept it. 



#### This is the end of the Skill lab 3. 

Copyright &copy; 2024 Marina Barsky.